# Conditional probability (Bayes) approach on informed sampling

## Some libraries

In [2]:
import sys
import logging

## LOGGING
nblog = open("alibaba_data-evaluation.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [3]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

In [4]:
import dask.array as da
import dask.distributed


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
from scipy.stats import norm
import numpy as np

import matplotlib as mpl
from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [7]:
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax

In [10]:
import hdbscan
from sklearn.cluster import OPTICS

In [11]:
from collections import defaultdict
from collections import Counter

In [12]:
import random

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

In [14]:
import shap

In [15]:
from xgboost import plot_importance

from xgboost import to_graphviz

from sklearn.metrics import classification_report,confusion_matrix

In [16]:
import pickle

## Load the dataset

In [17]:
sampled_workload_data = pd.read_csv('experiments/100_001_sampled_workload_data_full.csv', index_col=0)

In [18]:
sampled_workload_data

,inst_id,user,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type,group,workload,inst_name,worker_name,machine_x,gpu_name,cpu_usage,gpu_wrk_util,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,read,write,read_count,write_count,duration,gpu_type_machine_spec,cap_cpu_machine_spec,cap_mem_machine_spec,cap_gpu_machine_spec,machine_y,machine_cpu_iowait_machine_metric,machine_cpu_kernel_machine_metric,machine_cpu_usr_machine_metric,machine_gpu_machine_metric,machine_load_1_machine_metric,machine_net_receive_machine_metric,machine_num_worker_machine_metric,machine_cpu_machine_metric
job_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71712e3c961b5bbdb2ce0440,c5be8813bf90aaf9ca6f02f744ccd95596e9850928a498...,2cbd33b08024,ps,5.0,Terminated,4625696.0,4628399.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,c2e0be7577f427c47d77dd388f5bad181447f0bd21bc9e...,04e8a1d93c8a93a3a0a10466cd2f25132dd14c73293d24...,137e86ce1bc508e64d5fc5ee,/dev/nvidia1,34.465789,7.541667,1.204708,2.723633,0.347158,0.829102,2.068066e+06,3.426616e+05,312.218182,219.376623,2703.0,P100,64.0,512.0,2.0,137e86ce1bc508e64d5fc5ee,0.005480,2.603109,9.889830,45.808696,7.668093,2.620698e+08,5.335775,12.498419
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,1e999417385ceac6fc6908fc833dbc1bf7ccb5ab9e1af7...,3c41010cedf0cc85aa05e197131d20a5684be2c7dd2844...,e43590f20ce91e21cf8f0062,/dev/nvidia0,52.794326,12.049123,1.505760,2.671875,0.445045,0.827148,1.234028e+06,3.724553e+05,264.849650,173.374126,1595.0,P100,64.0,512.0,2.0,e43590f20ce91e21cf8f0062,0.001801,10.529087,28.466644,80.292014,34.026547,1.068209e+08,3.888263,38.997531
b68f25260271d65caccca8e7,5229275f7f92865554f934e97f576ef99db0c079bba59d...,d4d51aca8806,worker,16.0,Terminated,4058070.0,4058321.0,50.0,29.296875,100.0,T4,8bc721404a5d34c9f68829312d324994,bert,f004b4f39329b0c12a6b8de546f95fe477885ed004c856...,db23db95b2af54b7a37b4cc1beb4935c57a5c385d654a8...,f1f4f45a9474a2fd362d76fe,/dev/nvidia0,75.052632,23.727273,1.037900,1.853516,0.445890,0.774414,1.010761e+07,3.781242e+06,2401.913043,590.652174,251.0,T4,96.0,512.0,2.0,f1f4f45a9474a2fd362d76fe,0.002509,2.282917,1.616539,28.132630,4.036872,1.746246e+07,1.537805,3.901965
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,570f01d774b50c6d0026967d83b1c3403d093ffbe4c474...,3c8e0b3564bf056fbd705402c5d3429e8a89643d4c8d6c...,3c4af9c9b5db2d0a07860338,/dev/nvidia1,92.604240,11.706294,1.481600,2.667969,0.449140,0.829102,2.358233e+06,4.527896e+05,404.843206,271.797909,1595.0,P100,64.0,512.0,2.0,3c4af9c9b5db2d0a07860338,0.002109,1.872507,6.680456,20.784838,5.452154,1.350479e+08,5.147730,8.555072
446cd4921e0f00ab5ce1c2ea,12b9ec70cc7c63ebc7184b8a0c363513a8c3cb19746c74...,2cbd33b08024,ps,2.0,Terminated,5521030.0,5526746.0,600.0,20.000000,NaN,NaN,4ac7f9c48a6913dfa81169f7d6a150fa,bert,a5d7f3cf14cc3baef1b59eab27895e2222f2302735d873...,c34f6c58302ec92032293a469d2cbfaccba65ff97203b6...,d4f54f7e5cf18341942c47ef,/dev/nvidia0,110.590788,13.005300,2.357810,3.056641,0.645605,0.827148,1.037695e+06,1.527555e+05,201.449250,181.084731,5716.0,P100,64.0,512.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e49cae3b33b5c205ed8542b0,5283a27d4f19d767256f97c18880a70d69f4d97ca655a3...,47baba33cffe,ps,1.0,Terminated,5890601.0,5893675.0,600.0,29.296875,NaN,NaN,7389796d26bc0d9c8bae9700a46e88ca,xlnet,8d6b9f3b9ef8539217bc16388fb88e18542c0048677344...,20fb53a63dab2ac5da0e2bc08328ee3835d5b0bf0a78f2...,2988f6d00a08192b50b767f4,/dev/nvidia1,127.340568,81.609635,1.940527,1.995117,1.200031,1.213867,1.941028e+06,1.165604e+06,402.442786,171.301824,

## Load hdbscan results

In [19]:
hdbscan_clusterer = pickle.load(open("experiments/hdbscan_300_power_transform_euclidean.pkl", "rb"))

In [20]:
label_stats = Counter(hdbscan_clusterer.labels_)

label_stats.most_common(10)


[(-1, 24603),
 (8, 12522),
 (9, 11612),
 (25, 7754),
 (23, 7474),
 (1, 5751),
 (12, 5501),
 (3, 3043),
 (13, 2287),
 (20, 2128)]

In [21]:
sampled_workload_data_with_profiles = sampled_workload_data.copy()
sampled_workload_data_with_profiles['profile'] = hdbscan_clusterer.labels_
sampled_workload_data_with_profiles
sampled_workload_data_with_profiles['profile'].value_counts()

-1     24603
 8     12522
 9     11612
 25     7754
 23     7474
 1      5751
 12     5501
 3      3043
 13     2287
 20     2128
 0      2101
 19     2072
 24     1994
 16     1943
 22     1891
 2       836
 15      777
 7       737
 5       735
 4       717
 14      686
 10      648
 18      585
 17      567
 21      395
 6       340
 11      302
Name: profile, dtype: int64

## Includes "cluster" -1 in the data BUT there is no "cluster -1" intelligent sampling.

In [22]:
sampled_workload_gpu_v100 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'V100']
sampled_workload_gpu_v100_count = sampled_workload_gpu_v100.groupby('profile').count()

In [23]:
V100_series = sampled_workload_gpu_v100_count['inst_id']
V100_series = V100_series.reindex(range(-1,26),fill_value=0)

In [24]:
sampled_workload_gpu_v100_count['inst_id']

profile
-1     493
 0      47
 1      74
 7       4
 8     316
 9     348
 10      9
 11      4
 12     69
 14     11
 17     37
 18      1
 19     21
 20     19
 21     13
 22      7
 23    108
 24      4
 25    138
Name: inst_id, dtype: int64

In [25]:
V100_series = sampled_workload_gpu_v100_count['inst_id']
V100_series = V100_series.reindex(range(-1,26),fill_value=0)
V100_series = V100_series.rename('V100')

In [26]:
sampled_workload_gpu_p100 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'P100']
sampled_workload_gpu_p100_count = sampled_workload_gpu_p100.groupby('profile').count()

In [27]:
P100_series = sampled_workload_gpu_p100_count['inst_id']
P100_series = P100_series.reindex(range(-1,26),fill_value=0)
P100_series = P100_series.rename('P100')

In [28]:
sampled_workload_gpu_misc = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'MISC']
sampled_workload_gpu_misc_count = sampled_workload_gpu_misc.groupby('profile').count()

In [29]:
misc_series = sampled_workload_gpu_misc_count['inst_id']
misc_series = misc_series.reindex(range(-1,26),fill_value=0)
misc_series = misc_series.rename('MISC')

In [30]:
sampled_workload_gpu_V100M32 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'V100M32']
sampled_workload_gpu_V100M32_count = sampled_workload_gpu_V100M32.groupby('profile').count()

In [31]:
V100M32_series = sampled_workload_gpu_V100M32_count['inst_id']
V100M32_series = V100M32_series.reindex(range(-1,26),fill_value=0)
V100M32_series = V100M32_series.rename('V100M32')

In [32]:
sampled_workload_gpu_T4 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'T4']
sampled_workload_gpu_T4_count = sampled_workload_gpu_T4.groupby('profile').count()

In [33]:
T4_series = sampled_workload_gpu_T4_count['inst_id']
T4_series = T4_series.reindex(range(-1,26),fill_value=0)
T4_series = T4_series.rename('T4')

In [34]:
sampled_workload_gpu_cpu = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'CPU']
sampled_workload_gpu_cpu_count = sampled_workload_gpu_cpu.groupby('profile').count()

In [35]:
CPU_series = sampled_workload_gpu_cpu_count['inst_id']
CPU_series = CPU_series.reindex(range(-1,26),fill_value=0)
CPU_series = CPU_series.rename('CPU')

In [36]:
gpu_profiles = pd.DataFrame([V100_series, P100_series, misc_series, V100M32_series, T4_series, CPU_series])

### P(profile | gpu)

In [37]:
gpu_profiles

profile,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
V100,493,47,74,0,0,0,0,0,4,316,348,9,4,69,0,11,0,0,37,1,21,19,13,7,108,4,138
P100,4118,26,0,0,0,0,0,0,89,7644,7590,298,0,0,0,283,0,0,220,97,680,624,137,151,1046,174,1060
MISC,12658,30,0,0,0,0,0,0,626,3636,3594,269,232,0,3,311,0,0,262,170,1234,1255,221,1722,6249,1796,6482
V100M32,250,0,0,0,0,0,0,0,2,11,8,3,0,0,0,1,0,0,4,11,11,10,1,2,15,3,18
T4,6501,1991,5677,836,3043,717,735,340,0,864,7,0,64,5432,2284,0,777,1943,0,297,0,74,0,0,0,0,0
CPU,576,6,0,0,0,0,0,0,16,50,64,69,2,0,0,79,0,0,44,9,125,145,23,9,54,16,54


In [38]:
gpus_profiles_prob = gpu_profiles.copy()
gpus_profiles_prob.loc['V100'] = gpus_profiles_prob.loc['V100']/gpus_profiles_prob.loc['V100'].sum()
gpus_profiles_prob.loc['P100'] = gpus_profiles_prob.loc['P100']/gpus_profiles_prob.loc['P100'].sum()
gpus_profiles_prob.loc['MISC'] = gpus_profiles_prob.loc['MISC']/gpus_profiles_prob.loc['MISC'].sum()
gpus_profiles_prob.loc['V100M32'] = gpus_profiles_prob.loc['V100M32']/gpus_profiles_prob.loc['V100M32'].sum()
gpus_profiles_prob.loc['T4'] = gpus_profiles_prob.loc['T4']/gpus_profiles_prob.loc['T4'].sum()
gpus_profiles_prob.loc['CPU'] = gpus_profiles_prob.loc['CPU']/gpus_profiles_prob.loc['CPU'].sum()

### P(profile | gpu)

In [39]:
gpus_profiles_prob

profile,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
V100,0.286129,0.027278,0.042948,0.000000,0.000000,0.000000,0.000000,0.000000,0.002322,0.183401,0.201973,0.005223,0.002322,0.040046,0.000000,0.006384,0.000000,0.000000,0.021474,0.000580,0.012188,0.011027,0.007545,0.004063,0.062681,0.002322,0.080093
P100,0.169906,0.001073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003672,0.315386,0.313158,0.012295,0.000000,0.000000,0.000000,0.011676,0.000000,0.000000,0.009077,0.004002,0.028056,0.025746,0.005653,0.006230,0.043157,0.007179,0.043735
MISC,0.310626,0.000736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015362,0.089227,0.088196,0.006601,0.005693,0.000000,0.000074,0.007632,0.000000,0.000000,0.006429,0.004172,0.030282,0.030798,0.005423,0.042258,0.153350,0.044074,0.159067
V100M32,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005714,0.031429,0.022857,0.008571,0.000000,0.000000,0.000000,0.002857,0.000000,0.000000,0.011429,0.031429,0.031429,0.028571,0.002857,0.005714,0.042857,0.008571,0.051429
T4,0.205845,0.063042,0.179754,0.026471,0.096352,0.022703,0.023273,0.010766,0.000000,0.027357,0.000222,0.000000,0.002026,0.171997,0.072320,0.000000,0.024603,0.061522,0.000000,0.009404,0.000000,0.002343,0.000000,0.000000,0.000000,0.000000,0.000000
CPU,0.429530,0.004474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011931,0.037286,0.047726,0.051454,0.001491,0.000000,0.000000,0.058911,0.000000,0.000000,0.032811,0.006711,0.093214,0.108128,0.017151,0.006711,0.040268,0.011931,0.040268


In [40]:
gpus_profiles_prob.loc['V100'].sum()

1.0

### P(gpu) 
From their article

In [44]:
gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}
gpus = pd.DataFrame.from_dict(gpus_dict, orient='index')
gpus = gpus.rename(columns={0:'gpu_counts'})

### P(gpu) 

In [45]:
gpus['prob'] = gpus['gpu_counts']/gpus.gpu_counts.sum()
gpus

,gpu_counts,prob
P100,798,0.420664
T4,497,0.261993
MISC,280,0.147601
V100M32,135,0.071165
V100,104,0.054823
CPU,83,0.043753


### P(gpu) 
From their article

### P(profile)

In [51]:
profiles = pd.DataFrame.from_dict(label_stats, orient='index').sort_index()
profiles = profiles.rename(columns={0:'profile_counts'})

In [ ]:
# profiles['prob'] = profiles['profile_counts']/profiles.profile_counts.sum()
# profiles

### P(profile)

In [47]:
for ii in profiles.index:
    profiles.loc[ii,'prob'] = (gpus_profiles_prob[ii]*gpus['prob']).sum()
# P_0 = (gpus_profiles_prob[0]*gpus['prob']).sum()
profiles

,profile_counts,prob
-1,24603,0.256564
0,2101,0.018768
1,5751,0.049449
2,836,0.006935
3,3043,0.025244
4,717,0.005948
5,735,0.006097
6,340,0.002821
7,737,0.004868
8,12522,0.166932


## GPU given a profile

In [ ]:
gpu_given_profile = gpus_profiles_prob.copy()
gpu_given_profile.loc['V100'] = gpus_profiles_prob.loc['V100']*gpus.loc['V100', 'prob']/profiles['prob']
gpu_given_profile.loc['P100'] = gpus_profiles_prob.loc['P100']*gpus.loc['P100', 'prob']/profiles['prob']
gpu_given_profile.loc['MISC'] = gpus_profiles_prob.loc['MISC']*gpus.loc['MISC', 'prob']/profiles['prob']
gpu_given_profile.loc['V100M32'] = gpus_profiles_prob.loc['V100M32']*gpus.loc['V100M32', 'prob']/profiles['prob']
gpu_given_profile.loc['T4'] = gpus_profiles_prob.loc['T4']*gpus.loc['T4', 'prob']/profiles['prob']
gpu_given_profile.loc['CPU'] = gpus_profiles_prob.loc['CPU']*gpus.loc['CPU', 'prob']/profiles['prob']

gpu_given_profile

In [ ]:
gpu_given_profile[24].sum()

In [ ]:
gpu_given_profile[20].plot(kind='bar')